# 1.  load data from kaggle  with item infomation 

In [63]:
indata1 ='archive/item_properties_part1.csv'
indata2 ='archive/item_properties_part2.csv'
df1 = pd.read_csv(indata1)
df2 = pd.read_csv(indata2)
item_properties=pd.concat([df1, df2])
item_properties=item_properties[item_properties['property']=='categoryid']
item_properties.drop(['property'], axis=1, inplace=True)
item_properties.drop(['timestamp'], axis=1, inplace=True)


In [64]:
new_data = []

for id in unique_item_id:
    # Select rows where itemid equals the current id
    filtered_rows = item_properties[item_properties['itemid'] == id]
    
    # Check if filtered_rows is empty
    if filtered_rows.empty:
        values = None  # Or you can use [] for an empty list
    else:
        # Check the number of values and format accordingly
        if len(filtered_rows) > 1:
            values = filtered_rows['value'].tolist()  # Convert values to a list if there are multiple
        else:
            values = filtered_rows['value'].iloc[0]  # Get the single value

    # Append the result as a dictionary
    new_data.append({'itemid': id, 'value': values})

# Convert new_data to a DataFrame
new_data_df = pd.DataFrame(new_data)

In [65]:
new_data_df

,itemid,value
0,43511,1179
1,54408,209
2,10006,1694
3,49432,1280
4,39563,1196
...,...,...
70847,12116,1255
70848,43011,1397
70849,14255,479
70850,29598,1287


In [4]:

import pandas as pd
parent= pd.read_csv('archive/category_tree.csv')
parent.head()


,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


# 2.  creat two lists of unique cat id and partent id for matrix columns

In [146]:
unique_category_id = parent['categoryid'].unique().tolist()
unique_parent_id = parent['parentid'].unique().tolist()

In [28]:
len(unique_category_id)


1669

In [ ]:
new_data_df

,itemid,categoryid,parentid
0,43511,1179,580.0
1,54408,209,293.0
2,10006,1694,1621.0
3,49432,1280,605.0
4,39563,1196,1181.0
...,...,...,...
70847,12116,1255,587.0
70848,43011,1397,1416.0
70849,14255,479,520.0
70850,29598,1287,1009.0


In [133]:
new_data_df= pd.read_csv('cat_parent.csv')
df=new_data_df
df

,itemid,categoryid,parentid
0,43511,1179,580.0
1,54408,209,293.0
2,10006,1694,1621.0
3,49432,1280,605.0
4,39563,1196,1181.0
...,...,...,...
70847,12116,1255,587.0
70848,43011,1397,1416.0
70849,14255,479,520.0
70850,29598,1287,1009.0


In [20]:
df.sample(10)

,itemid,categoryid,parentid
22986,3386,1400,335.0
9035,41206,184,1009.0
56047,20096,360,1174.0
42909,59697,65,293.0
15750,55496,NaN,NaN
11227,7053,1410,1495.0
5369,27030,508,362.0
38906,33197,342,500.0
8379,70193,"['1277', '1277', '1277', '1277', '1277', '1277...","[312.0, 312.0, 312.0, 312.0, 312.0, 312.0, 312..."
24658,68296,459,1330.0


In [32]:
df.dtypes

itemid         int64
categoryid    object
parentid      object
dtype: object

In [134]:
df['itemid'].astype('int64')

0        43511
1        54408
2        10006
3        49432
4        39563
         ...  
70847    12116
70848    43011
70849    14255
70850    29598
70851    46971
Name: itemid, Length: 70852, dtype: int64

# 3. creat blank one-hot encoded dataframe for both parentid and categoryid

In [135]:
# Initialize the one_hot_df for the entire DataFrame
one_hot_df = pd.DataFrame(index=df['itemid'].unique(), columns=unique_category_id)


In [148]:
one_hot_df = pd.DataFrame(index=df['itemid'].unique(), columns= unique_parent_id  )

# 4. map the info from original data to the one-hot encoded dataframe

In [149]:
# Define the update_one_hot function
def update_one_hot(row):
    item_id = row.itemid
    categories = row.parentid

    # Handle NaN values
    if pd.isna(categories):
        return

    # If the category is a string representation of a list, convert it to an actual list
    if isinstance(categories, str) and categories.startswith('[') and categories.endswith(']'):
        try:
            categories = ast.literal_eval(categories)
        except ValueError:
            print(f"Error parsing categories for item_id {item_id}: {categories}")
            return

    # Ensure categories is a list
    if not isinstance(categories, list):
        categories = [categories]

    # Get unique category IDs
    unique_categories = set(categories)
    for category in unique_categories:
        # int
        category = float(category)
        # category = int(category)
        if category in one_hot_df.columns:
            one_hot_df.at[item_id, category] = 1

# # Apply the function to each row in df
df.apply(update_one_hot, axis=1)



0        None
1        None
2        None
3        None
4        None
         ... 
70847    None
70848    None
70849    None
70850    None
70851    None
Length: 70852, dtype: object

In [150]:
# one_hot_df fillna 0
one_hot_df.fillna(0, inplace=True)

# 5. check if there is unmatched parentid or categoryid and drop them


In [151]:
one_hot_df = one_hot_df.astype(int)
columns_to_drop = one_hot_df.columns[(one_hot_df == 0).all()]
# one_hot_df.drop(columns=columns_to_drop, inplace=True)
columns_to_drop

Index([ 169.0,  885.0,    nan,  542.0, 1537.0,  711.0,  488.0,  232.0,  880.0,
        666.0, 1291.0,  865.0,  585.0,  399.0,  178.0, 1682.0,  571.0, 1370.0,
        609.0, 1444.0, 1394.0,  216.0,  369.0, 1532.0, 1396.0, 1156.0, 1579.0,
        359.0,  989.0, 1559.0,  696.0, 1211.0,  663.0, 1630.0, 1075.0,  479.0,
         55.0, 1107.0,  803.0,  334.0, 1168.0,  736.0,  657.0,  791.0,  431.0,
       1335.0,   48.0, 1120.0,  245.0,   95.0,  557.0, 1525.0,  157.0, 1401.0,
       1469.0, 1224.0, 1443.0,  132.0, 1160.0,  750.0,  659.0, 1057.0,  673.0,
         19.0, 1582.0,  513.0, 1452.0, 1182.0, 1288.0,  675.0, 1692.0, 1198.0,
       1315.0,  664.0, 1671.0, 1092.0,  785.0, 1327.0,  813.0, 1661.0,  731.0,
       1654.0],
      dtype='float64')

In [152]:
one_hot_df.drop(columns=columns_to_drop, inplace=True)

In [153]:
one_hot_df

,213.0,9.0,1691.0,378.0,1621.0,893.0,312.0,622.0,1281.0,1110.0,...,1081.0,799.0,1505.0,903.0,1157.0,385.0,778.0,1228.0,341.0,47.0
43511,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10006,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [155]:
# check again
one_hot_df = one_hot_df.astype(int)
columns_to_drop = one_hot_df.columns[(one_hot_df == 0).all()]
columns_to_drop

Index([], dtype='float64')

# 6. save the one-hot encoded dataframe to csv file

In [156]:
# save this file
one_hot_df.to_csv('one_hot_feature2.csv', index=False)